In [2]:
from transformers import pipeline
import datasets

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import config
import torch.nn.functional as F

In [5]:
import random
import pandas as pd
from IPython.display import display, HTML
import numpy as np
import os

In [23]:
#Training data

train = pd.read_csv('train.csv')
test = pd.read_csv('100test.csv')
print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

# First few rows of the training dataset
train.head()


Training data shape:  (104, 6)
Testing data shape:  (100, 1)


,created_at,favorite_count,id,full_text,reply_count,url
0,2021-09-20T05:47:45.000Z,51,1439828607510196225,Why aren’t all our universities now providing ...,161,https://twitter.com/BarrySheerman/status/14398...
1,2021-12-17T11:26:34.000Z,491,1471804006393540610,Some personal news: \n\nAfter 12 yrs at Twitte...,58,https://twitter.com/PratPanc/status/1471804006...
2,2021-09-19T08:37:15.000Z,151,1439508874818736130,England's universities must take student views...,36,https://twitter.com/BBCNews/status/14395088748...
3,2021-10-01T08:13:49.000Z,799,1443851631376977921,"Over 1,800 undergraduates at Warwick Universit...",22,https://twitter.com/UsforThemUK/status/1443851...
4,2021-09-10T15:58:48.000Z,79,1436358503971659781,Quite right of Educ' Sec @GavinWilliamson to...,21,https://twitter.com/toryboypierce/status/14363...


In [24]:
# First few rows of the testing dataset
test.head()

,full_text
0,Just posted a photo @ Missouri Western State U...
1,So with the numbers of LAW STUDENTS in univers...
2,It's so hard to make friends after university!...
3,Former University of Kansas professor found gu...
4,Congratulations to one of our high flying scho...


In [25]:
#Missing values in training set
train.isnull().sum()


created_at        0
favorite_count    0
id                0
full_text         0
reply_count       0
url               0
dtype: int64

In [26]:
#Missing values in test set
test.isnull().sum()

full_text    0
dtype: int64

In [27]:
# Dropping missing values
test.dropna(axis = 0, how ='any',inplace=True) ;

In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

这一段代码将从HuggingFace的Transformer库下载预训练好的情感分析模型。
https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment
这个预训练模型它以多颗星（1到5颗星之间）预测评论的情绪。

In [29]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("techthiyanes/chinese_sentiment")

model = AutoModelForSequenceClassification.from_pretrained("techthiyanes/chinese_sentiment")

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

lables = ['Negative', 'Neutral', 'Positive']

五颗星情感分类

tokens = tokenizer.encode('I hated this, absolutely the worst', return_tensors='pt')

tokens

tokenizer.decode(tokens[0])

result = model(tokens)
result

int(torch.argmax(result.logits))+1


In [30]:
tokens = tokenizer.encode('我爱你', return_tensors='pt')
result = model(tokens)
print(int(torch.argmax(result.logits))+1)

5


In [31]:
df = pd.DataFrame(test, columns=['full_text'])

In [32]:
print(df)

                                            full_text
0   Just posted a photo @ Missouri Western State U...
1   So with the numbers of LAW STUDENTS in univers...
2   It's so hard to make friends after university!...
3   Former University of Kansas professor found gu...
4   Congratulations to one of our high flying scho...
..                                                ...
95  I've just seen a SMH article suggesting that, ...
96  @bridgett65679 @FoxNews Where did get your PhD...
97  @JNU_official_50 @EduMinOfIndia I am deeply so...
98  You’re all invited to my graduation ceremony ✨...
99  @Shashan15378205 @AskAnshul Wow..... You gives...

[100 rows x 1 columns]


In [33]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, padding=True, 
                            truncation = True, max_length=50, return_tensors='pt') 
    result = model(tokens)
    return int(torch.argmax(result.logits))+1



In [34]:
sentiment_score(df['full_text'].iloc[0])


1

In [35]:
df['sentiment'] = df['full_text'].apply(lambda x: sentiment_score(x[:512]))

In [36]:
df

,full_text,sentiment
0,Just posted a photo @ Missouri Western State U...,1
1,So with the numbers of LAW STUDENTS in univers...,3
2,It's so hard to make friends after university!...,5
3,Former University of Kansas professor found gu...,1
4,Congratulations to one of our high flying scho...,5
...,...,...
95,"I've just seen a SMH article suggesting that, ...",2
96,@bridgett65679 @FoxNews Where did get your PhD...,3
97,@JNU_official_50 @EduMinOfIndia I am deeply so...,1
98,You’re all invited to my graduation ceremony ✨...,5


In [37]:
df.to_csv("101test.csv")

In [20]:
df['sentiment'].value_counts()

1    39
5    28
4    20
3    12
2     1
Name: sentiment, dtype: int64

In [21]:
df['sentiment'].value_counts(normalize=True)

1    0.39
5    0.28
4    0.20
3    0.12
2    0.01
Name: sentiment, dtype: float64

三维情感分类

In [214]:
tweet = 'You are my becon 😄'

In [215]:
#precprcess tweet
tweet_words = []

for word in tweet.split(' '):
    if word.startswith('@') and len(word) > 1:
        word = '@user'
    elif word.startswith('http'):
        word = 'http'
    tweet_words.append(word)

tweet_proc = ' '.join(tweet_words)
#print(tweet_words)
#print(tweet_proc)

In [216]:
# load model and tokenizer
roberta = 'cardiffnlp/twitter-roberta-base-sentiment'
model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)
lables = ['Negative', 'Neutral', 'Positive']

In [217]:
# sentiment analysis
encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
#print(encoded_tweet)

In [218]:
output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
#print(output)

In [219]:
output = model(**encoded_tweet)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

for i in range(len(scores)):
    l = lables[i]
    s = scores[i]
    print(l, s)
#print(scores)

Negative 0.0044955024
Neutral 0.12450922
Positive 0.8709953
